# Databricks Job Pipeline – Task level parameter

This example demonstrates:
- How to **set a parameter value** in one notebook using `dbutils.jobs.taskValues.set()`
- How to **retrieve the value** in another notebook using `dbutils.jobs.taskValues.get()`
- Use case: We filter transaction data (`txns`) by `state` and `city` parameters passed between notebooks.

**Dataset Fields:**
`txnid, txndate, custid, amount, prodcategory, prodname, city, state, paymenttype`

**Flow:**
1. The parent notebook defines parameters (`state`, `city`)  
2. Reads sample `txns` data  
3. Filters data and sets a **record count** value using `dbutils.jobs.taskValues.set()`  
4. The child notebook retrieves this value and performs further logic  

In [0]:
txns_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("dbfs:/Volumes/inceptez_catalog/inputdb/customerdata/txns") \
	.toDF("txnid","txndate","custid","amount","prodcategory","prodname","city","state","paymenttype")

In [0]:
state = "Texas"
city = "Dallas"

from pyspark.sql import functions as F

filtered_df = txns_df.filter((txns_df.state == state) & (txns_df.city == city))
df1 = filtered_df.withColumn("created_ts",F.current_timestamp())
df1.write.format("delta").mode("append").saveAsTable("inceptez_catalog.outputdb.tbltxnsjob")
print("data written into table")

In [0]:
# Store the record_count value to be used in another notebook
dbutils.jobs.taskValues.set(
    key="txn_record_count",
    value=record_count
)

print("Record count stored successfully in job task values.")
